## <span style='color:red;'>Code scapping</span>

### <span style='color:blue;'>Amazone Scrapping </span>

Ce code est un script Python conçu pour scraper des données de produits à partir de plusieurs URLs d'Amazon France (https://www.amazon.fr/). Il utilise les bibliothèques requests pour effectuer des requêtes HTTP, BeautifulSoup pour analyser le contenu HTML des pages, et pandas pour organiser les données collectées dans un DataFrame. Le script commence par définir des en-têtes HTTP pour simuler une requête provenant d'un navigateur web, puis parcourt une liste d'URLs spécifiques à des catégories de produits (ordinateurs portables, smartphones, airpods, smartwatch). Pour chaque URL, il extrait des informations telles que le titre du produit, le prix, la note moyenne, le nombre d'avis, les promotions éventuelles et la disponibilité en stock. Ces données sont nettoyées et stockées dans une liste, puis converties en un DataFrame Pandas pour une manipulation et une analyse ultérieures. Le script inclut également des pauses entre les requêtes pour éviter d'être bloqué par le site. Enfin, il affiche les données collectées sous forme de tableau. 
Des nettoyages supplémentaires pourraient être effectués ultérieurement.

In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Définition de l'User-Agent et des en-têtes
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.amazon.fr/',
    'Connection': 'keep-alive'
}

# Liste des URLs des sites à scraper
urls = [
    'https://www.amazon.fr/s?k=ordinateur+portable',
    'https://www.amazon.fr/s?k=smartphone',
    'https://www.amazon.fr/s?k=smartwatch',
    'https://www.amazon.fr/s?k=airpods'
]

# Stockage des résultats
all_data = []

# Fonction pour scraper une URL donnée
def scrape_url(url):
    try:
        response = requests.get(url, headers=headers, timeout=100)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Détection du site cible
        if "amazon.fr" in url:
            site = "Amazon"
            items = soup.select('.s-result-item')

            for item in items:
                title = item.select_one('div.a-section.a-spacing-none.a-spacing-top-small.s-title-instructions-style > a > h2 > span')
                price = item.select_one('span.a-offscreen')
                rating = item.select_one('span.a-icon-alt')
                reviews = item.select_one('span.a-size-base.s-underline-text')
                stock = item.select_one('span.a-size-base.a-color-price')
                promo = item.select_one('span.a-price.a-text-price > span:nth-child(2)')

                # Nettoyage des données
                title_text = title.text.strip() if title else None
                price_text = price.text.strip().replace('€', '').replace(',', '.') if price else 0
                rating_text = rating.text.split()[0].replace(',', '.') if rating else 0
                reviews_text = reviews.text.replace('(', '').replace(')', '').replace(',', '') if reviews else 0
                promo_text = promo.text.replace('€', '').replace(',', '.') if promo else 0
                stock_text = stock.text.strip()if stock else 'Disponible'
               

                # Ajout des données à la liste
                if title_text and price_text:
                    all_data.append({
                        'Site': site,
                        'Titre': title_text,
                        'Prix (€)': price_text,
                        'Note': rating_text,
                        'Avis': reviews_text,
                        'promo': promo_text,
                        'Stock': stock_text
                    })
        else:
            print(f"⚠️ Site non pris en charge : {url}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors de la requête {url}: {e}")

    time.sleep(2)  # Pause pour éviter le blocage

# Scraping de toutes les URLs
for url in urls:
    scrape_url(url)

# Conversion des résultats en DataFrame Pandas
ama = pd.DataFrame(all_data)

# Affichage du DataFrame
ama

,Site,Titre,Prix (€),Note,Avis,promo,Stock
0,Amazon,Asus CX1500CKA-NJ0400 15.6 Pouces FHD PC Porta...,249.99,4.2,29,449.99,Disponible
1,Amazon,"ACER Chromebook 315 CB315-4H-C7R0, Ordinateur ...",219.00,4.2,342,299.00,Disponible
2,Amazon,LG Gram Superslim 15Z90ST-G.AA78F - PC Portabl...,1 638.99,3.8,43,1 899.99,Il ne reste plus que 10 exemplaire(s) en stock...
3,Amazon,"Ordinateur Portable 16 Pouces,Core I5 Processe...",369.99,5.0,2,369.99,Disponible
4,Amazon,ACER Aspire 1 A115-32-C3AK Ordinateur Portable...,249.00,4.3,1 147,299.00,Disponible
...,...,...,...,...,...,...,...
208,Amazon,"Sudio A1 Violet Écouteurs avec Bluetooth, Touc...",40.99,3.6,8,0,Disponible
209,Amazon,CMF by Nothing Buds Pro 2 Écouteurs sans Fil a...,59.00,4.5,1 261,0,Disponible
210,Amazon,Soundcore Ecouteurs Bluetooth sans Fil P40i by...,66.30,4.3,3 191,69.99,Disponible
211,Amazon,"JBL Endurance Peak 3, Écouteurs de Sport sans ...",84.99,4.2,2 100,99.99,Disponible


In [2]:
# Sauvegarde en CSV
ama.to_csv("resultats_scrapama196.csv", index=False)

### <span style='color:blue;'>Boulanger Scrapping </span>

<p>Nous avons adopter une approche de séparation des scripts par produits compte tenue des erreurs rencontrées lors du scrapping!</p>
Ce script Python scrape des données de produits (smartphones, aipods, smartwarch, laptops) sur le site de Boulanger en envoyant des requêtes HTTP et en analysant le HTML avec BeautifulSoup. Il extrait des informations comme le titre, le prix, la note, les avis, les promotions et la disponibilité des produits, puis nettoie et structure ces données dans une liste. Ensuite, il les convertit en un DataFrame Pandas pour une visualisation facile et les sauvegarde dans un fichier CSV

#### <span style='color:blue;'>Smartphone </span>

In [14]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup


# Définition de l'User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

url = 'https://www.boulanger.com/c/smartphone-telephone-portable',

# Stockage des résultats
all_data = []

# Fonction de scraping
def scrape_product(url):
    try:
        response = requests.get(url, headers=headers, timeout=900)
        response.raise_for_status()  # Vérification des erreurs HTTP
        soup = BeautifulSoup(response.content, 'html.parser')

        
     
        items = soup.select(f'#product-list > ul > li > article')#main > div.grid.product-list__content
            
        len(items)
        for  item in items :
                title = item.select_one('#productLabel')
                price= item.select_one('.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > div > p')
                rating = item.select_one('.product-list__product-area-2.g-col-5.g-col-sm-7.g-col-md-4.g-col-lg-3.g-col-xl-3 > div.product-list__product-rating > a > div > bl-rating')
                reviews = item.select_one('.product-list__product-area-2.g-col-5.g-col-sm-7.g-col-md-4.g-col-lg-3.g-col-xl-3 > div.product-list__product-rating > a > span')
                promo = item.select_one('.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > div > div > span.price__crossed')
                stock = item.select_one('.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > button > span')

                if title and price:
                    all_data.append({
                        'Titre': title.text.strip(),
                        'Prix (€)': price.text.replace(',', '.').strip() if price else None,
                        'Note': rating.get('rating') if rating else None,
                        'Avis': reviews.text.strip() if reviews else None,  # Boulanger ne montre pas toujours le nombre d’avis
                        'promo': promo.text.strip() if promo else "0",
                        'Stock' : 'Disponible' if stock else 'Indisponible'
                        })
                    
        print(len(items))          


    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors de la requête : {e}")

    time.sleep(2)  # Pause pour éviter le blocage

# Exécution du scraping
for url in url:
        scrape_product(url)

# Conversion en DataFrame Pandas pour visualisation
boultel = pd.DataFrame(all_data)

# Affichage des résultats
boultel

50


,Titre,Prix (€),Note,Avis,promo,Stock
0,Smartphone\n\t\t\t\t\t\t\tSAMSUNG\n\t\t\t\t\t\...,1471.99€,5.0,(5),0,Disponible
1,Smartphone\n\t\t\t\t\t\t\tSAMSUNG\n\t\t\t\t\t\...,301.99€,4.692299842834473,(13),"351,04€",Disponible
2,Smartphone\n\t\t\t\t\t\t\tSAMSUNG\n\t\t\t\t\t\...,901.99€,None,None,0,Disponible
3,Smartphone\n\t\t\t\t\t\t\tHONOR\n\t\t\t\t\t\t\...,301.99€,None,None,"381,99€",Disponible
4,Smartphone\n\t\t\t\t\t\t\tXIAOMI\n\t\t\t\t\t\t...,461.99€,5.0,(1),"501,99€",Disponible
5,Smartphone\n\t\t\t\t\t\t\tXIAOMI\n\t\t\t\t\t\t...,299.99€,4.028600215911865,(35),"371,06€",Disponible
6,Smartphone\n\t\t\t\t\t\t\tGOOGLE\n\t\t\t\t\t\t...,429.00€,4.1875,(16),0,Disponible
7,Smartphone\n\t\t\t\t\t\t\tSAMSUNG\n\t\t\t\t\t\...,502.04€,4.550000190734863,(40),0,Disponible
8,Smartphone\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\...,399.00€,3.75,(12),"449,00€",Disponible
9,Smartphone\n\t\t\t\t\t\t\tSAMSUNG\n\t\t\t\t\t\...,988.90€,4.933300018310547,(30),0,Disponible


#### <span style='color:blue;'>Laptops</span>

In [ ]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup


# Définition de l'User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

url = 'https://www.boulanger.com/c/tous-les-ordinateurs-portables',

# Stockage des résultats
all_data = []

# Fonction de scraping
def scrape_product(url):
    try:
        response = requests.get(url, headers=headers, timeout=900)
        response.raise_for_status()  # Vérification des erreurs HTTP
        soup = BeautifulSoup(response.content, 'html.parser')

        
     
        items = soup.select(f'#product-list > ul > li > article')#main > div.grid.product-list__content
            
        len(items)
        for  item in items :
                title = item.select_one('#productLabel')
                price= item.select_one('.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > div > p')
                rating = item.select_one('.product-list__product-area-2.g-col-5.g-col-sm-7.g-col-md-4.g-col-lg-3.g-col-xl-3 > div.product-list__product-rating > a > div > bl-rating')
                reviews = item.select_one('.product-list__product-area-2.g-col-5.g-col-sm-7.g-col-md-4.g-col-lg-3.g-col-xl-3 > div.product-list__product-rating > a > span')
                promo = item.select_one('.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > div > div > span.price__crossed')
                stock = item.select_one('.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > button > span')

                if title and price:
                    all_data.append({
                        'Titre': title.text.strip(),
                        'Prix (€)': price.text.replace(',', '.').strip() if price else None,
                        'Note': rating.get('rating') if rating else None,
                        'Avis': reviews.text.strip() if reviews else None,  # Boulanger ne montre pas toujours le nombre d’avis
                        'promo': promo.text.strip() if promo else "0",
                        'Stock' : 'Disponible' if stock else 'Indisponible'
                        })
                    
        print(len(items))            


    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors de la requête : {e}")

    time.sleep(2)  # Pause pour éviter le blocage

# Exécution du scraping
for url in url:
        scrape_product(url)

# Conversion en DataFrame Pandas pour visualisation
boulpc = pd.DataFrame(all_data)

# Affichage des résultats
boulpc

50

📊 Résultats du scraping :
                                                Titre  Prix (€)  \
0   Ordinateur portable\n\t\t\t\t\t\t\tLENOVO\n\t\...   699.99€   
1   Ordinateur portable\n\t\t\t\t\t\t\tLENOVO\n\t\...   429.99€   
2   Ordinateur portable\n\t\t\t\t\t\t\tASUS\n\t\t\...   699.99€   
3   PC Gamer\n\t\t\t\t\t\t\tLENOVO\n\t\t\t\t\t\t\t...   799.99€   
4   Ordinateur portable\n\t\t\t\t\t\t\tLENOVO\n\t\...   449.99€   
5   PC Hybride\n\t\t\t\t\t\t\tLENOVO\n\t\t\t\t\t\t...   649.99€   
6   PC Gamer\n\t\t\t\t\t\t\tASUS\n\t\t\t\t\t\t\tTU...   999.99€   
7   Ordinateur portable\n\t\t\t\t\t\t\tLENOVO\n\t\...   499.99€   
8   Ordinateur portable\n\t\t\t\t\t\t\tHP\n\t\t\t\...   649.99€   
9   PC Hybride\n\t\t\t\t\t\t\tHP\n\t\t\t\t\t\t\tEn...   899.99€   
10  PC Hybride\n\t\t\t\t\t\t\tHP\n\t\t\t\t\t\t\tPa...   899.99€   
11  PC Gamer\n\t\t\t\t\t\t\tMSI\n\t\t\t\t\t\t\tThi...   699.99€   
12  Ordinateur Apple\n\t\t\t\t\t\t\tMACBOOK\n\t\t\...  1299.00€   
13  Ordinateur portable\n\t\t\t\

#### <span style='color:blue;'>Airpods</span>

In [ ]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup


# Définition de l'User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

url = 'https://www.boulanger.com/c/airpods',

# Stockage des résultats
all_data = []

# Fonction de scraping
def scrape_product(url):
    try:
        response = requests.get(url, headers=headers, timeout=900)
        response.raise_for_status()  # Vérification des erreurs HTTP
        soup = BeautifulSoup(response.content, 'html.parser')

        
     
        items = soup.select(f'#product-list > ul > li > article')#main > div.grid.product-list__content
            
        len(items)
        for  item in items :
                title = item.select_one('#productLabel')
                price= item.select_one('.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > div > p')
                rating = item.select_one('.product-list__product-area-2.g-col-5.g-col-sm-7.g-col-md-4.g-col-lg-3.g-col-xl-3 > div.product-list__product-rating > a > div > bl-rating')
                reviews = item.select_one('.product-list__product-area-2.g-col-5.g-col-sm-7.g-col-md-4.g-col-lg-3.g-col-xl-3 > div.product-list__product-rating > a > span')
                promo = item.select_one('.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > div > div > span.price__crossed')
                stock = item.select_one('.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > button > span')

                if title and price:
                    all_data.append({
                        'Titre': title.text.strip(),
                        'Prix (€)': price.text.replace(',', '.').strip() if price else None,
                        'Note': rating.get('rating') if rating else None,
                        'Avis': reviews.text.strip() if reviews else None,  # Boulanger ne montre pas toujours le nombre d’avis
                        'promo': promo.text.strip() if promo else "0",
                        'Stock' : 'Disponible' if stock else 'Indisponible'
                        })
                    
        print(len(items))      


    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors de la requête : {e}")

    time.sleep(2)  # Pause pour éviter le blocage

# Exécution du scraping
for url in url:
        scrape_product(url)

# Conversion en DataFrame Pandas pour visualisation
boulair = pd.DataFrame(all_data)

# Affichage des résultats
boulair


44

📊 Résultats du scraping :
                                                Titre Prix (€)  \
0   Ecouteurs\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\t...  149.00€   
1   Casque\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\tAir...  579.00€   
2   Ecouteurs\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\t...  279.00€   
3   Casque\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\tAir...  579.00€   
4   Ecouteurs\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\t...  199.00€   
5   Casque\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\tAir...  579.00€   
6   Casque\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\tAir...  579.00€   
7   Casque\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\tAir...  579.00€   
8   Ecouteurs\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\t...  149.99€   
9   Casque\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\tAir...  549.00€   
10  Ecouteurs\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\t...  129.99€   
11  Ecouteurs\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\t...   19.54€   
12  Ecouteurs\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\t...   19.03€   
13  Casque\n\t\t\t\t\t\t\tAPPLE\n\t\t\t\t\t\t\

#### <span style='color:blue;'>Smartwatch</span>

In [ ]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup


# Définition de l'User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

url = 'https://www.boulanger.com/c/montre-connectee',

# Stockage des résultats
all_data = []

# Fonction de scraping
def scrape_product(url):
    try:
        response = requests.get(url, headers=headers, timeout=90)
        response.raise_for_status()  # Vérification des erreurs HTTP
        soup = BeautifulSoup(response.content, 'html.parser')

        
     
        items = soup.select(f'#product-list > ul > li > article')#main > div.grid.product-list__content
            
        len(items)
        for  item in items :
                title = item.select_one('#productLabel')
                price= item.select_one('.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > div > p')
                rating = item.select_one('.product-list__product-area-2.g-col-5.g-col-sm-7.g-col-md-4.g-col-lg-3.g-col-xl-3 > div.product-list__product-rating > a > div > bl-rating')
                reviews = item.select_one('.product-list__product-area-2.g-col-5.g-col-sm-7.g-col-md-4.g-col-lg-3.g-col-xl-3 > div.product-list__product-rating > a > span')
                promo = item.select_one('.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > div > div > span.price__crossed')
                stock = item.select_one('.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > button > span')

                if title and price:
                    all_data.append({
                        'Titre': title.text.strip(),
                        'Prix (€)': price.text.replace(',', '.').strip() if price else None,
                        'Note': rating.get('rating') if rating else None,
                        'Avis': reviews.text.strip() if reviews else None,  # Boulanger ne montre pas toujours le nombre d’avis
                        'promo': promo.text.strip() if promo else "0",
                        'Stock' : 'Disponible' if stock else 'Indisponible'
                        })
                    
        print(len(items))             


    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors de la requête : {e}")

    time.sleep(2)  # Pause pour éviter le blocage

# Exécution du scraping
for url in url:
        scrape_product(url)

# Conversion en DataFrame Pandas pour visualisation
boulwatch = pd.DataFrame(all_data)

# Affichage des résultats
boulwatch


50

📊 Résultats du scraping :


### <span style='color:blue;'>Jumia Nigeria Scrapping </span>

Ce script Python automatisé effectue du web scraping sur Jumia Nigeria pour extraire des informations sur des produits spécifiques (smartwatches, smartphones, AirPods, laptops) en utilisant les bibliothèques requests et BeautifulSoup. Il navigue vers les URLs de recherche définies, simule une requête d'utilisateur via des en-têtes HTTP personnalisés, puis analyse le contenu HTML pour collecter le titre, le prix, la note, les avis, la disponibilité en stock, et les rabais. Les données sont ensuite nettoyées, organisées dans un DataFrame Pandas.

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Définition de l'User-Agent et des en-têtes
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.jumia.com.ng/',
    'Connection': 'keep-alive'
}

# Liste des URLs des sites à scraper
urls = [
    'https://www.jumia.com.ng/catalog/?q=smartwatch',
    'https://www.jumia.com.ng/catalog/?q=smartphone',
    'https://www.jumia.com.ng/catalog/?q=airpods',
    'https://www.jumia.com.ng/catalog/?q=laptops'
]

# Stockage des résultats
all_data = []

# Fonction pour scraper une URL donnée
def scrape_url(url):
    try:
        response = requests.get(url, headers=headers, timeout=1000)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Détection du site cible
        if "jumia.com.ng" in url:
            site = "jumia"
            items = soup.select('#jm > main > div.aim.row.-pbm > div.-pvs.col12 > section > div > article')

            for item in items:
                # Extraction des données
                title = item.select_one('h3')
                price = item.select_one('div.prc')
                rating = item.select_one('div.rev > div')
                reviews = item.select_one('div.rev')
                discount = item.select_one('div.bdg._dsct._sm') 
                # Nettoyage des données
                title_text = title.text.strip() if title else "N/A"
                price_text = price.text.strip() if price else "N/A"
                rating_text = rating.text.strip() if rating else "N/A"
                reviews_text = reviews.text.strip() if reviews else "0"
                stock_text = "Non renseigné"
                discount_text = discount.text.strip() if discount else "Aucun rabais"  


                # Ajout des données à la liste
                if title_text and price_text:
                    all_data.append({
                        'Site': site,
                        'Titre': title_text,
                        'Prix (Naira)': price_text,
                        'Note': rating_text.replace('out of 5', ' '),
                        'Avis': reviews_text,
                        'Stock': stock_text,
                        'Rabais': discount_text  # Ajout du rabais
                    })
        else:
            print(f"⚠️ Site non pris en charge : {url}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors de la requête {url}: {e}")

    time.sleep(2)  # Pause pour éviter le blocage

# Scraping de toutes les URLs
for url in urls:
    scrape_url(url)

# Conversion des résultats en DataFrame Pandas
jumia = pd.DataFrame(all_data)

# Affichage du DataFrame
jumia


,Site,Titre,Prix (Naira),Note,Avis,Stock,Rabais
0,jumia,TiLECC T800 Smart Watch Ultra 8 Door Access S...,"₦ 10,599",3.5,3.5 out of 5(1674),Non renseigné,40%
1,jumia,XS8 Pro Ultra Smartwatch Bluetooth Phone Water...,"₦ 15,990",3.8,3.8 out of 5(42),Non renseigné,40%
2,jumia,Y68 Bluetooth Smart Connected Fitness Digital ...,"₦ 6,000",3.9,3.9 out of 5(14),Non renseigné,20%
3,jumia,Series 8 Fitness I20 Ultra Max SUIT 10 In 1 Se...,"₦ 34,800",4.6,4.6 out of 5(12),Non renseigné,1%
4,jumia,Series 8 Smart Watch 8 Ultra 45mm 49mm TWO Str...,"₦ 25,000 - ₦ 26,000",3.1,3.1 out of 5(27),Non renseigné,26%
...,...,...,...,...,...,...,...
155,jumia,Wooden Beside Laptop Desk Bamboo Adjustable La...,"₦ 22,999",N/A,0,Non renseigné,Aucun rabais
156,jumia,Ace Elec 14.1'' Intel Celeron J4105 4Core CPU ...,"₦ 224,289",N/A,0,Non renseigné,Aucun rabais
157,jumia,Portable Aluminum 7~15.4 Inch Laptop Stand Als...,"₦ 13,500",4.2,4.2 out of 5(6),Non renseigné,Aucun rabais
158,jumia,Adjustable Folding Laptop Stand Desk 4 PC Note...,"₦ 28,400",4.6,4.6 out of 5(25),Non renseigné,Aucun rabais


### <span style='color:blue;'> Materiel.net Scrapping </span>

In [55]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_materiel(url):
    """Scraping de la page Materiel"""
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        items = soup.select('.c-products-list__item')
        for item in items:
            # Sélecteurs spécifiques à Materiel
            title = item.select_one('.c-product__title')
            price = item.select_one('.o-product__price')
            description = item.select_one('.c-product__description')
            Dispo = item.select_one('.o-availability__value')
            
            title_text = title.text.strip() if title else None
            price_text = price.text.strip().replace('€', '.').replace(' ', '') if price else None
            description_text = description.text if description else None
            Dispo_text = Dispo.text if Dispo else "0"
            
            if title_text and price_text:
                if "smartphone" in url:
                    all_data.append({
                        'Site': 'Materiel.net',                                        
                        'Titre': title_text,
                        'Prix_euro': price_text,
                        'Note' : 0 ,
                        'Avis' : 0 ,
                        'Promo': 0 ,
                        'Stock': Dispo_text ,
                        'Produit' : 'Smartphone',
                        'Description': description_text
                    })
                elif "ordinateur" in url:
                    all_data.append({
                        'Site': 'Materiel.net',                                        
                        'Titre': title_text,
                        'Prix_euro': price_text,
                        'Note' : 0 ,
                        'Avis' : 0 ,
                        'Promo': 0 ,
                        'Stock': Dispo_text ,
                        'Produit' : 'Laptop',
                        'Description': description_text
                    })
                elif "smartwatch" in url:
                    all_data.append({
                        'Site': 'Materiel.net',                                        
                        'Titre': title_text,
                        'Prix_euro': price_text,
                        'Note' : 0 ,
                        'Avis' : 0 ,
                        'Promo': 0 ,
                        'Stock': Dispo_text ,
                        'Produit' : 'Smartwatch',
                        'Description': description_text
                    })
                elif "airpods" in url:
                    all_data.append({
                        'Site': 'Materiel.net',                                        
                        'Titre': title_text,
                        'Prix_euro': price_text,
                        'Note' : 0 ,
                        'Avis' : 0 ,
                        'Promo': 0 ,
                        'Stock': Dispo_text ,
                        'Produit' : 'AirPods',
                        'Description': description_text
                    })
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors du scraping sur Amazon : {e}")
    time.sleep(2)

all_data = []
urls = [
    'https://www.materiel.net/recherche/smartphone/',
    'https://www.materiel.net/recherche/ordinateur%20portable/',
    'https://www.materiel.net/recherche/smartwatch/',
    'https://www.materiel.net/recherche/airpods/'
]
for url in urls:
    scrape_materiel(url)
# Conversion des résultats en DataFrame Pandas
mat = pd.DataFrame(all_data)

# Affichage du DataFrame
mat

,Site,Titre,Prix_euro,Note,Avis,Promo,Stock,Produit,Description
0,Materiel.net,Samsung Galaxy S25 5G (Vert d'eau) - 256 Go,959.95,0,0,0,En stock,Smartphone,"Galaxy S25 5G, Vert d'eau, 256 Go, 6,2 pouces,..."
1,Materiel.net,Samsung Galaxy S25 Ultra 5G (Bleu) - 1 To,1 829.95,0,0,0,En stock,Smartphone,"Galaxy S25 Ultra 5G, Bleu, 1 To, 6,9 pouces, A..."
2,Materiel.net,Samsung Galaxy S25 Ultra 5G (Bleu) - 512 Go,1 589.95,0,0,0,En stock,Smartphone,"Galaxy S25 Ultra 5G, Bleu, 512 Go, 6,9 pouces,..."
3,Materiel.net,Samsung Galaxy S25 Ultra 5G (Gris) - 256 Go,1 469.95,0,0,0,En stock,Smartphone,"Galaxy S25 Ultra 5G, Gris, 256 Go, 6,9 pouces,..."
4,Materiel.net,Samsung Galaxy S25 5G (Bleu nuit) - 128 Go,899.95,0,0,0,En stock,Smartphone,"Galaxy S25 5G, Bleu nuit, 128 Go, 6,2 pouces, ..."
...,...,...,...,...,...,...,...,...,...
171,Materiel.net,Belkin Chargeur Magsafe 15W + chargeur pour Ap...,129.95,0,0,0,En stock,AirPods,Station de recharge 2-en-1 Stand avec MagSafe ...
172,Materiel.net,Apple Câble USB-C vers Lightning (2024) - 1 m,25.00,0,0,0,En stock,AirPods,Câble de chargement et synchronisation pour iP...
173,Materiel.net,Apple Câble USB-C vers Lightning - 2 m,35.00,0,0,0,En stock,AirPods,Câble de chargement et synchronisation pour iP...
174,Materiel.net,Apple AirPods Max Gris sidéral - Casque sans fil,579.00,0,0,0,Rupture,AirPods,"Casque à réduction active de bruit, HiFi, assi..."


### <span style='color:blue;'>LDLC Scrapping </span>

Ce code est un script de scraping web qui extrait des informations sur différents produits (smartphones, ordinateurs, smartwatches et airpods) à partir du site LDLC. Il utilise les bibliothèques `requests` pour effectuer des requêtes HTTP, `BeautifulSoup` pour parser le contenu HTML des pages, et `pandas` pour organiser les données récupérées dans un DataFrame. Le script parcourt une liste d'URLs spécifiques à chaque catégorie de produits, extrait des détails tels que le titre, le prix, la description et la disponibilité de chaque produit, puis stocke ces informations dans une liste. Enfin, cette liste est convertie en un DataFrame Pandas pour une manipulation et une analyse ultérieures. Le script inclut également une gestion des erreurs pour les requêtes HTTP et une pause de 2 secondes entre chaque requête pour éviter de surcharger le serveur.

In [53]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_ldlc(url):
    """Scraping de la page ldlc"""
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        items = soup.select('.pdt-item')
        for item in items:
            # Sélecteurs spécifiques à ldlc
            title = item.select_one('.title-3')
            price = item.select_one('.new-price')
            if not price :
                price = item.select_one('.price')
            description = item.select_one('.desc')
            Dispo = item.select_one('.stock')
            
            title_text = title.text.strip() if title else None
            price_text = price.text.strip().replace('€', '.').replace(' ', '') if price else None
            description_text = description.text if description else None
            Dispo_text = Dispo.text if Dispo else "0"
            
            if title_text and price_text:
                if "smartphone" in url:
                    all_data.append({
                        'Site': 'ldlc',                                        
                        'Titre': title_text,
                        'Prix_euro': price_text,
                        'Note' : 0 ,
                        'Avis' : 0 ,
                        'Promo': 0 ,
                        'Stock': Dispo_text ,
                        'Produit' : 'Smartphone',
                        'Description': description_text
                    })
                elif "ordinateur" in url:
                    all_data.append({
                        'Site': 'ldlc',                                        
                        'Titre': title_text,
                        'Prix_euro': price_text,
                        'Note' : 0 ,
                        'Avis' : 0 ,
                        'Promo': 0 ,
                        'Stock': Dispo_text ,
                        'Produit' : 'Laptop',
                        'Description': description_text
                    })
                elif "smartwatch" in url:
                    all_data.append({
                        'Site': 'ldlc',                                        
                        'Titre': title_text,
                        'Prix_euro': price_text,
                        'Note' : 0 ,
                        'Avis' : 0 ,
                        'Promo': 0 ,
                        'Stock': Dispo_text ,
                        'Produit' : 'Smartwatch',
                        'Description': description_text
                    })
                elif "airpods" in url:
                    all_data.append({
                        'Site': 'ldlc',                                        
                        'Titre': title_text,
                        'Prix_euro': price_text,
                        'Note' : 0 ,
                        'Avis' : 0 ,
                        'Promo': 0 ,
                        'Stock': Dispo_text ,
                        'Produit' : 'AirPods',
                        'Description': description_text
                    })
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors du scraping sur Amazon : {e}")
    time.sleep(2)

all_data = []
urls = [
    'https://www.ldlc.com/recherche/smartphone/',
    'https://www.ldlc.com/recherche/ordinateur/',
    'https://www.ldlc.com/recherche/smartwatch/',
    'https://www.ldlc.com/recherche/airpods/'
]
for url in urls:
    scrape_ldlc(url)
# Conversion des résultats en DataFrame Pandas
ldlc = pd.DataFrame(all_data)

# Affichage du DataFrame
ldlc

,Site,Titre,Prix_euro,Note,Avis,Promo,Stock,Produit,Description
0,ldlc,Samsung Galaxy S25 Ultra SM-S938B Bleu Titane ...,1 829.95,0,0,0,En stock,Smartphone,Smartphone 5G-LTE Dual SIM IP68 avec Galaxy AI...
1,ldlc,Samsung Galaxy S25 Ultra SM-S938B Bleu Titane ...,1 589.95,0,0,0,En stock,Smartphone,Smartphone 5G-LTE Dual SIM IP68 avec Galaxy AI...
2,ldlc,Samsung Galaxy S25 SM-S931B Bleu Nuit (12 Go /...,959.95,0,0,0,En stock,Smartphone,Smartphone 5G-LTE Dual SIM IP68 avec Galaxy AI...
3,ldlc,Samsung Galaxy S25 SM-S931B Bleu Clair (12 Go ...,1 079.95,0,0,0,En stock,Smartphone,Smartphone 5G-LTE Dual SIM IP68 avec Galaxy AI...
4,ldlc,Samsung Galaxy S25+ SM-S936B Bleu Nuit (12 Go ...,1 169.95,0,0,0,En stock,Smartphone,Smartphone 5G-LTE Dual SIM IP68 avec Galaxy AI...
...,...,...,...,...,...,...,...,...,...
187,ldlc,Coque Airpods Silicone Bubble Pop Conception 2...,7.96,0,0,0,En stock,AirPods,"Coque Bubble pop Fidget Toy, conçue pour le bo..."
188,ldlc,Coque AirPods Pro / Pro 2 Silicone Bubble Pop ...,7.96,0,0,0,En stock,AirPods,"Coque Bubble pop Fidget Toy, conçue pour le bo..."
189,ldlc,Coque Airpods Bubble Pop Silicone Conception 2...,7.96,0,0,0,En stock,AirPods,"Coque Bubble pop Fidget Toy, conçue pour le bo..."
190,ldlc,Coque AirPods Pro et Pro 2 Bubble Pop Silicone...,7.96,0,0,0,En stock,AirPods,"Coque Bubble pop Fidget Toy, conçue pour le bo..."


### <span style='color:blue;'>Quelques traitements suplémentaires</span>

In [50]:
def nettoyer_base_de_donnees1(df):

   
    # 1. Renommer les colonnes pour une meilleure lisibilité
    df.columns = ['Site','Titre', 'Prix_euro', 'Note', 'Avis', 'Promo', 'Stock']
    

    # Convertir les colonnes numériques
    df['Prix_euro'] = (df['Prix_euro'].astype(str).str.replace('\u202f', '').str.replace('\xa0', '').astype(float))
    df['Note'] = df['Note'].astype(float)
    df['Avis'] = df['Avis'].astype(str).str.replace('\xa0', '').astype(int)
    df['Promo'] = ((df['Promo'].astype(str).str.replace('\u202f', '').str.replace('\xa0', '').astype(float))- df['Prix_euro'])/(df['Promo'].astype(str).str.replace('\u202f', '').str.replace('\xa0', '').astype(float))


    df = df[['Site', 'Titre', 'Prix_euro', 'Note', 'Avis', 'Promo', 'Stock']] 

    return df

amazone = nettoyer_base_de_donnees1(ama)

# Fonction pour classifier les produits
def classify_product1(title):
    title_lower = str(title).lower()  # Convertir en minuscules pour uniformiser la recherche
    if 'smart watch' in title_lower or 'smartwatch' in title_lower or 'montre' in title_lower or 'connectée' in title_lower:
        return 'Smartwatch'
    elif 'smartphone' in title_lower or 'telephone' in title_lower or 'redmi' in title_lower or 'sim' in title_lower or 'itel' in title_lower or 'zte' in title_lower:
        return 'Smartphone'
    elif 'airpods' in title_lower or 'écouteur' in title_lower or 'earphone' in title_lower or 'casque' in title_lower:
        return 'AirPods'
    elif 'laptop' in title_lower or 'book' in title_lower or 'pc' in title_lower or 'intel' in title_lower or 'ordinateur' in title_lower  or 'portable' in title_lower:
        return 'Laptop'
    else:
        return 'Autre'

# Ajout d'une nouvelle colonne "Type de Produit"
amazone['Produit'] = amazone['Titre'].apply(classify_product1)

In [51]:
amazone

,Site,Titre,Prix_euro,Note,Avis,Promo,Stock,Produit
0,Amazon,Asus CX1500CKA-NJ0400 15.6 Pouces FHD PC Porta...,249.99,4.2,29,0.444454,Disponible,Laptop
1,Amazon,"ACER Chromebook 315 CB315-4H-C7R0, Ordinateur ...",219.00,4.2,342,0.267559,Disponible,Laptop
2,Amazon,LG Gram Superslim 15Z90ST-G.AA78F - PC Portabl...,1638.99,3.8,43,0.137369,Il ne reste plus que 10 exemplaire(s) en stock...,Laptop
3,Amazon,"Ordinateur Portable 16 Pouces,Core I5 Processe...",369.99,5.0,2,0.000000,Disponible,Laptop
4,Amazon,ACER Aspire 1 A115-32-C3AK Ordinateur Portable...,249.00,4.3,1147,0.167224,Disponible,Laptop
...,...,...,...,...,...,...,...,...
208,Amazon,"Sudio A1 Violet Écouteurs avec Bluetooth, Touc...",40.99,3.6,8,-inf,Disponible,AirPods
209,Amazon,CMF by Nothing Buds Pro 2 Écouteurs sans Fil a...,59.00,4.5,1261,-inf,Disponible,AirPods
210,Amazon,Soundcore Ecouteurs Bluetooth sans Fil P40i by...,66.30,4.3,3191,0.052722,Disponible,Autre
211,Amazon,"JBL Endurance Peak 3, Écouteurs de Sport sans ...",84.99,4.2,2100,0.150015,Disponible,AirPods


In [ ]:
def nettoyer_base_de_donnees(df):

   
    # 1. Renommer les colonnes pour une meilleure lisibilité
    df.columns = ['Titre', 'Prix_euro', 'Note', 'Avis', 'Promo', 'Stock']
    

    # Convertir les colonnes numériques
    df['Prix_euro'] = df['Prix_euro'].astype(str).str.replace('€', '').str.replace(',', '.').astype(float)
    df['Note'] = df['Note'].astype(float)
    df['Avis'] = df['Avis'].astype(str).str.extract(r'\((\d+)\)').fillna(0).astype(int)
    if (df['Promo'] == 0) : 
        df['Promo'] = 0
    else :
        df['Promo'] = ((df['Promo'].astype(str).str.replace('€', '').str.replace(',', '.').astype(float)) - df['Prix_euro'])/(df['Promo'].astype(str).str.replace('€', '').str.replace(',', '.').astype(float))


    df['Site'] = 'Boulanger'

    df = df[['Site', 'Titre', 'Prix_euro', 'Note', 'Avis', 'Promo', 'Stock']] 

    return df

boultel = nettoyer_base_de_donnees(boultel)
boultel['Produit'] = 'Smartphone'

boulair = nettoyer_base_de_donnees(boulair)
boulair['Produit'] = 'Airpods'

boulpc = nettoyer_base_de_donnees(boulpc)
boulpc['Produit'] = 'Laptop'

boulwatch = nettoyer_base_de_donnees(boulwatch)
boulwatch['Produit'] = 'Smartwatch'


ValueError: Length mismatch: Expected axis has 8 elements, new values have 6 elements

In [25]:
import pandas as pd

# Concaténation des DataFrames
Boulanger = pd.concat([boultel, boulair, boulpc, boulwatch], ignore_index=True)

Boulanger

,Site,Titre,Prix_euro,Note,Avis,Promo,Stock,Produit
0,Boulanger,Smartphone\n\t\t\t\t\t\t\tSAMSUNG\n\t\t\t\t\t\...,1704.012424,5.0000,0,-inf,Disponible,Smartphone
1,Boulanger,Smartphone\n\t\t\t\t\t\t\tSAMSUNG\n\t\t\t\t\t\...,349.591174,4.6923,0,0.004127,Disponible,Smartphone
2,Boulanger,Smartphone\n\t\t\t\t\t\t\tSAMSUNG\n\t\t\t\t\t\...,1044.166174,NaN,0,-inf,Disponible,Smartphone
3,Boulanger,Smartphone\n\t\t\t\t\t\t\tHONOR\n\t\t\t\t\t\t\...,349.591174,NaN,0,0.084816,Disponible,Smartphone
4,Boulanger,Smartphone\n\t\t\t\t\t\t\tXIAOMI\n\t\t\t\t\t\t...,534.811174,5.0000,0,-0.065382,Disponible,Smartphone
...,...,...,...,...,...,...,...,...
189,Boulanger,Montre santé\n\t\t\t\t\t\t\tSAMSUNG\n\t\t\t\t\...,69.990000,3.6667,6,-inf,Disponible,Smartwatch
190,Boulanger,Montre sport\n\t\t\t\t\t\t\tGARMIN\n\t\t\t\t\t...,149.990000,4.8000,10,-inf,Disponible,Smartwatch
191,Boulanger,Montre sport\n\t\t\t\t\t\t\tGARMIN\n\t\t\t\t\t...,429.990000,NaN,0,-inf,Disponible,Smartwatch
192,Boulanger,Montre sport\n\t\t\t\t\t\t\tGARMIN\n\t\t\t\t\t...,270.040000,4.3226,31,-inf,Disponible,Smartwatch


In [31]:
# Nettoyage des données
# 1. Renommer les colonnes pour une meilleure lisibilité
jumia.columns = ['Site', 'Titre', 'Prix_euro', 'Note', 'Avis', 'Stock', 'Promo']


# 2. Nettoyer la colonne "Prix_Naira" pour extraire les valeurs numériques
jumia['Prix_euro'] = jumia['Prix_euro'].astype(str).str.replace('₦', '').str.replace('e', '').str.replace(',', '').str.strip()

# Function to handle ranges 
def convert_price(price):
    if '-' in price:
        prices = price.split('-')
        prices = [float(p.strip()) for p in prices]
        return sum(prices) /2
    else:
        return price

# Apply the function to the column
jumia['Prix_euro'] = (jumia['Prix_euro'].apply(convert_price).astype(float)) * 0.00063

# 3. Convertir la colonne "Note" en float
jumia['Note'] = jumia['Note'].replace('N/A', 0).fillna(0).astype(float)

# 4. Extraire le nombre d'avis depuis la colonne "Avis"
jumia['Avis'] = jumia['Avis'].str.extract(r'\((\d+)\)').fillna(0).astype(int)

# 5. Normaliser la colonne "Stock" 
jumia['Stock'] = jumia['Stock']

# 6. Nettoyer la colonne "Rabais" pour extraire les pourcentages
jumia['Promo'] = (jumia['Promo'].str.replace('%', '').str.replace('Aucun rabais', '0').str.strip().astype(float))*0.01


jumia = jumia[['Site', 'Titre', 'Prix_euro', 'Note', 'Avis', 'Promo', 'Stock']] 


# Fonction pour classifier les produits
def classify_product(title):
    title_lower = str(title).lower()  # Convertir en minuscules pour uniformiser la recherche
    if 'smart watch' in title_lower or 'smartwatch' in title_lower:
        return 'Smartwatch'
    elif 'smartphone' in title_lower or 'android' in title_lower or 'redmi' in title_lower or 'sim' in title_lower or 'itel' in title_lower or 'zte' in title_lower:
        return 'Smartphone'
    elif 'airpods' in title_lower or 'pods' in title_lower or 'earphone' in title_lower or 'ear' in title_lower:
        return 'AirPods'
    elif 'laptop' in title_lower or 'book' in title_lower or 'pc' in title_lower or 'intel' in title_lower or 'dell' in title_lower or 'hp' in title_lower:
        return 'Laptop'
    else:
        return 'Autre'

# Ajout d'une nouvelle colonne "Type de Produit"
jumia['Produit'] = jumia['Titre'].apply(classify_product)


C:\Users\ACER\AppData\Local\Temp\ipykernel_5572\1273461003.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jumia['Produit'] = jumia['Titre'].apply(classify_product)


In [32]:
jumia

,Site,Titre,Prix_euro,Note,Avis,Promo,Stock,Produit
0,jumia,TiLECC T800 Smart Watch Ultra 8 Door Access S...,0.002431,3.5,1674,0.40,Non renseigné,Smartwatch
1,jumia,XS8 Pro Ultra Smartwatch Bluetooth Phone Water...,0.002515,3.8,42,0.40,Non renseigné,Smartwatch
2,jumia,Y68 Bluetooth Smart Connected Fitness Digital ...,0.002359,3.9,14,0.20,Non renseigné,Smartwatch
3,jumia,Series 8 Fitness I20 Ultra Max SUIT 10 In 1 Se...,0.002809,4.6,12,0.01,Non renseigné,Smartwatch
4,jumia,Series 8 Smart Watch 8 Ultra 45mm 49mm TWO Str...,0.002664,3.1,27,0.26,Non renseigné,Smartwatch
...,...,...,...,...,...,...,...,...
155,jumia,Wooden Beside Laptop Desk Bamboo Adjustable La...,0.002625,0.0,0,0.00,Non renseigné,Laptop
156,jumia,Ace Elec 14.1'' Intel Celeron J4105 4Core CPU ...,0.002553,0.0,0,0.00,Non renseigné,Laptop
157,jumia,Portable Aluminum 7~15.4 Inch Laptop Stand Als...,0.002476,4.2,6,0.00,Non renseigné,Laptop
158,jumia,Adjustable Folding Laptop Stand Desk 4 PC Note...,0.002709,4.6,25,0.00,Non renseigné,Laptop


In [56]:
# Concaténation des DataFrames
df = pd.concat([jumia, Boulanger, amazone,  ldlc, mat], ignore_index=True)

In [57]:
df

,Site,Titre,Prix_euro,Note,Avis,Promo,Stock,Produit,Description
0,jumia,TiLECC T800 Smart Watch Ultra 8 Door Access S...,0.002431,3.5,1674,0.40,Non renseigné,Smartwatch,NaN
1,jumia,XS8 Pro Ultra Smartwatch Bluetooth Phone Water...,0.002515,3.8,42,0.40,Non renseigné,Smartwatch,NaN
2,jumia,Y68 Bluetooth Smart Connected Fitness Digital ...,0.002359,3.9,14,0.20,Non renseigné,Smartwatch,NaN
3,jumia,Series 8 Fitness I20 Ultra Max SUIT 10 In 1 Se...,0.002809,4.6,12,0.01,Non renseigné,Smartwatch,NaN
4,jumia,Series 8 Smart Watch 8 Ultra 45mm 49mm TWO Str...,0.002664,3.1,27,0.26,Non renseigné,Smartwatch,NaN
...,...,...,...,...,...,...,...,...,...
930,Materiel.net,Belkin Chargeur Magsafe 15W + chargeur pour Ap...,129.95,0.0,0,0.00,En stock,AirPods,Station de recharge 2-en-1 Stand avec MagSafe ...
931,Materiel.net,Apple Câble USB-C vers Lightning (2024) - 1 m,25.00,0.0,0,0.00,En stock,AirPods,Câble de chargement et synchronisation pour iP...
932,Materiel.net,Apple Câble USB-C vers Lightning - 2 m,35.00,0.0,0,0.00,En stock,AirPods,Câble de chargement et synchronisation pour iP...
933,Materiel.net,Apple AirPods Max Gris sidéral - Casque sans fil,579.00,0.0,0,0.00,Rupture,AirPods,"Casque à réduction active de bruit, HiFi, assi..."


In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


# Sidebar for navigation
st.sidebar.title("Navigation")
page = st.sidebar.radio("Choisir une page", [
    "Analyse de la Concurrence",
    "Analyse des Promotions",
    "Analyse des Stocks",
    "Évolution des Prix",
    "Produits les Mieux Notés",
    "Produits avec le Plus d'Avis",
    "Comparaison des Prix"
])

# Valeurs par défaut pour Top/Bottom N
top_n = st.sidebar.slider("Sélectionner le nombre de produits pour Top/Bottom N", min_value=1, max_value=10, value=5)

# Page 1: Analyse de la Concurrence
if page == "Analyse de la Concurrence":
    st.title("📊 Analyse de la Concurrence entre Sites")

    # Value Box : Nombre total de produits
    st.metric(label="Nombre Total de Produits", value=len(df))

        # Tableau Récapitulatif : Prix moyen par site
    avg_prices = df.groupby('Site').agg({
        'Titre' : 'count',
        'Prix_euro': 'mean',  
        'Avis': 'mean',       
        'Note': 'mean'       
    }).reset_index()

    # Renommer les colonnes pour une meilleure lisibilité
    avg_prices.columns = ['Site', 'Total produit', 'Prix Moyen euro', 'Nombre Moyen d\'Avis', 'Note Moyenne']

    # Afficher le tableau récapitulatif
    st.write("### Tableau Récapitulatif : Prix Moyen, Nombre Moyen d'Avis et Note Moyenne par Site")
    st.table(avg_prices.style.format({
        'Total produit': '{:.2f}',
        'Prix Moyen euro': '{:.2f}',  
        'Nombre Moyen d\'Avis': '{:.0f}',  
        'Note Moyenne': '{:.2f}'     
    }))

    # Ajouter des graphiques sur la même ligne
    st.write("### Comparaison Graphique entre les Sites")

    # Créer 4 colonnes pour afficher les graphiques côte à côte
    col1, col2, col3, col4 = st.columns(4)

    # Nuage de points 1 : Prix Moyen vs Total Produits
    with col1:
        fig1 = px.scatter(
            avg_prices,
            x='Total Produits',
            y='Prix Moyen euro',
            color='Site',
            title="Prix Moyen vs Total Produits",
            labels={'Prix Moyen euro': 'Prix Moyen (€)', 'Total Produits': 'Nombre de Produits'},
            hover_name='Site'
        )
        fig1.update_layout(width=350, height=350)
        st.plotly_chart(fig1)

    # Nuage de points 2 : Nombre Moyen d'Avis vs Prix Moyen
    with col2:
        fig2 = px.scatter(
            avg_prices,
            x='Prix Moyen euro',
            y='Nombre Moyen d\'Avis',
            color='Site',
            title="Nombre Moyen d'Avis vs Prix Moyen",
            labels={'Nombre Moyen d\'Avis': 'Nombre Moyen d\'Avis', 'Prix Moyen euro': 'Prix Moyen (€)'},
            hover_name='Site'
        )
        fig2.update_layout(width=350, height=350)
        st.plotly_chart(fig2)

    # Nuage de points 3 : Note Moyenne vs Nombre Moyen d'Avis
    with col3:
        fig3 = px.scatter(
            avg_prices,
            x='Nombre Moyen d\'Avis',
            y='Note Moyenne',
            color='Site',
            title="Note Moyenne vs Nombre Moyen d'Avis",
            labels={'Nombre Moyen d\'Avis': 'Nombre Moyen d\'Avis', 'Note Moyenne': 'Note Moyenne'},
            hover_name='Site'
        )
        fig3.update_layout(width=350, height=350)
        st.plotly_chart(fig3)

    # Nuage de points 4 : Total Produits vs Note Moyenne
    with col4:
        fig4 = px.scatter(
            avg_prices,
            x='Total Produits',
            y='Note Moyenne',
            color='Site',
            title="Note Moyenne vs Total Produits",
            labels={'Total Produits': 'Nombre de Produits', 'Note Moyenne': 'Note Moyenne'},
            hover_name='Site'
        )
        fig4.update_layout(width=350, height=350)
        st.plotly_chart(fig4)

    col1, col2 = st.columns(2)

    with col1 :
        # Top N et Bottom N produits par site
        st.write(f"### Top {top_n} Produits par Site")
        top_products = df.sort_values(by='Prix_euro', ascending=False).groupby('Site').head(top_n)
        st.table(top_products[['Site', 'Titre', 'Prix_euro', 'Produit']])
    with col2 :
        st.write(f"### Bottom {top_n} Produits par Site")
        bottom_products = df.sort_values(by='Prix_euro').groupby('Site').head(top_n)
        st.table(bottom_products[['Site', 'Titre', 'Prix_euro', 'Produit']])

# Page 2: Analyse des Promotions
elif page == "Analyse des Promotions":
    st.title("🎉 Analyse des Promotions")

    # Filtrer les produits en promotion
    promo_df = df[df['Promo'] > 0]

    # Value Box : Nombre total de produits en promotion
    st.metric(label="Nombre de Produits en Promotion", value=len(promo_df))

    # Tableau Récapitulatif : Taux de rabais moyen par site
    avg_promo = promo_df.groupby('Site')['Promo'].mean().reset_index()
    avg_promo.columns = ['Site', 'Taux de Rabais Moyen (%)']
    st.write("### Tableau Récapitulatif : Taux de Rabais Moyen par Site")
    st.table(avg_promo)

    # Graphique : Taux de rabais par produit
    fig_promo = px.bar(
        promo_df,
        x='Titre',
        y='Promo',
        color='Site',
        title=f"Taux de Rabais pour les Produits en Promotion (Top {top_n})",
        labels={'Promo': 'Taux de Rabais (%)', 'Titre': 'Produit'}
    )
    st.plotly_chart(fig_promo)
    col1, col2 = st.columns(2)
        # Top N et Bottom N produits en promotion
    with col1 :
        st.write(f"### Top {top_n} Produits en Promotion (Plus Grand Rabais)")
        top_promo = promo_df.sort_values(by='Promo', ascending=False).head(top_n)
        st.table(top_promo[['Site', 'Titre', 'Prix_euro', 'Promo']])
    with col2 : 
        st.write(f"### Bottom {top_n} Produits en Promotion (Plus Petit Rabais)")
        bottom_promo = promo_df.sort_values(by='Promo').head(top_n)
        st.table(bottom_promo[['Site', 'Titre', 'Prix_euro', 'Promo']])

# Page 3: Analyse des Stocks
elif page == "Analyse des Stocks":
    st.title("📦 Analyse des Stocks")

    # Value Box : Pourcentage de produits disponibles
    total_products = len(df)
    available_products = len(df[df['Stock'] == 'disponible'])
    unavailable_products = len(df[df['Stock'] == 'non disponible'])
    st.metric(label="Pourcentage de Produits Disponibles", value=f"{(available_products / total_products * 100):.2f}%")
    st.metric(label="Pourcentage de Produits Indisponibles", value=f"{(unavailable_products / total_products * 100):.2f}%")

    # Tableau Récapitulatif : Répartition des stocks par site
    stock_summary = df.groupby(['Site', 'Stock']).size().unstack(fill_value=0).reset_index()
    stock_summary.columns = ['Site', 'Disponible', 'Indisponible']
    st.write("### Tableau Récapitulatif : Répartition des Stocks par Site")
    st.table(stock_summary)

    # Graphique : Répartition des stocks par site
    fig_stock = px.bar(
        stock_summary.melt(id_vars=['Site'], value_vars=['Disponible', 'Indisponible']),
        x='Site',
        y='value',
        color='variable',
        title="Répartition des Stocks par Site",
        labels={'value': 'Nombre de Produits', 'variable': 'Statut du Stock'},
        barmode='group'
    )
    st.plotly_chart(fig_stock)

    col1, col2 = st.columns(2)
        # Top N et Bottom N produits selon leur disponibilité
    with col1 : 
        st.write(f"### Top {top_n} Produits Disponibles")
        top_available = df[df['Stock'] == 'disponible'].sort_values(by='Avis', ascending=False).head(top_n)
        st.table(top_available[['Site', 'Titre', 'Prix_euro', 'Stock', 'Avis']])
    with col2 :
        st.write(f"### Bottom {top_n} Produits Indisponibles")
        bottom_unavailable = df[df['Stock'] == 'non disponible'].sort_values(by='Avis').head(top_n)
        st.table(bottom_unavailable[['Site', 'Titre', 'Prix_euro', 'Stock', 'Avis']])

# Page 4: Évolution des Prix
elif page == "Évolution des Prix":
    st.title("📈 Évolution des Prix par Produit")

    # Sélectionner un produit spécifique
    selected_product = st.selectbox("Sélectionnez un produit :", df['Produit'].unique())
    product_data = df[df['Produit'] == selected_product]

    if not product_data.empty:
        # Value Box : Prix avant 
        mean_price = product_data['Prix_euro'].iloc[0]
        st.metric(label="Prix", value=f"€{mean_price:.2f}")



# Page 5: Produits les Mieux Notés
elif page == "Produits les Mieux Notés":
    st.title("🌟 Produits les Mieux Notés")

    # Value Box : Note moyenne globale
    avg_rating = df['Note'].mean()
    st.metric(label="Note Moyenne Globale", value=f"{avg_rating:.2f}")

    # Top N produits mieux notés
    st.write(f"### Top {top_n} Produits Mieux Notés")
    top_rated = df.sort_values(by='Note', ascending=False).head(top_n)
    st.table(top_rated[['Site', 'Titre', 'Prix_euro', 'Note', 'Avis']])

    # Bottom N produits moins bien notés
    st.write(f"### Bottom {top_n} Produits Moins Bien Notés")
    bottom_rated = df.sort_values(by='Note').head(top_n)
    st.table(bottom_rated[['Site', 'Titre', 'Prix_euro', 'Note', 'Avis']])

# Page 6: Produits avec le Plus d'Avis
elif page == "Produits avec le Plus d'Avis":
    st.title("💬 Produits avec le Plus d'Avis")

    # Value Box : Nombre total d'avis
    total_reviews = df['Avis'].sum()
    st.metric(label="Nombre Total d'Avis", value=total_reviews)

    # Top N produits avec le plus d'avis
    st.write(f"### Top {top_n} Produits avec le Plus d'Avis")
    top_reviews = df.sort_values(by='Avis', ascending=False).head(top_n)
    st.table(top_reviews[['Site', 'Titre', 'Prix_euro', 'Note', 'Avis']])

    # Bottom N produits avec le moins d'avis
    st.write(f"### Bottom {top_n} Produits avec le Moins d'Avis")
    bottom_reviews = df.sort_values(by='Avis').head(top_n)
    st.table(bottom_reviews[['Site', 'Titre', 'Prix_euro', 'Note', 'Avis']])

# Page 7: Comparaison des Prix
elif page == "Comparaison des Prix":
    st.title("💰 Comparaison des Prix entre Sites")

    # Value Box : Prix minimum et maximum
    min_price = df['Prix_euro'].min()
    max_price = df['Prix_euro'].max()
    st.metric(label="Prix Minimum", value=f"€{min_price:.2f}")
    st.metric(label="Prix Maximum", value=f"€{max_price:.2f}")

    # Tableau Récapitulatif : Prix par site
    price_summary = df.groupby('Site')['Prix_euro'].agg(['min', 'max', 'mean']).reset_index()
    price_summary.columns = ['Site', 'Prix Minimum (€)', 'Prix Maximum (€)', 'Prix Moyen (€)']
    st.write("### Tableau Récapitulatif : Prix par Site")
    st.table(price_summary)

    # Graphique : Boîtes à moustaches pour comparer les prix
    fig_box = px.box(
        df,
        x='Site',
        y='Prix_euro',
        title="Comparaison des Prix entre Sites",
        labels={'Prix_euro': 'Prix (€)', 'Site': 'Site'}
    )
    st.plotly_chart(fig_box)

    # Top N et Bottom N produits selon le prix
    st.write(f"### Top {top_n} Produits les Plus Chers")
    top_expensive = df.sort_values(by='Prix_euro', ascending=False).head(top_n)
    st.table(top_expensive[['Site', 'Titre', 'Prix_euro', 'Note', 'Avis']])

    st.write(f"### Bottom {top_n} Produits les Moins Chers")
    bottom_cheapest = df.sort_values(by='Prix_euro').head(top_n)
    st.table(bottom_cheapest[['Site', 'Titre', 'Prix_euro', 'Note', 'Avis']])

2025-02-18 14:19:14.202 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 14:19:15.026 
  command:

    streamlit run c:\Users\ACER\Desktop\webscraping-course-main\scraping_env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-18 14:19:15.028 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 14:19:15.029 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 14:19:15.033 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 14:19:15.035 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 14:19:15.038 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 14

NameError: name 'df' is not defined